In [1]:
import pandas as pd
import numpy as np
import re
import requests
from bs4 import BeautifulSoup
from itertools import zip_longest

In [5]:
# URL of the page
base_url = 'https://flatreviews.com/dubai/reviews/581?page='

# Create lists to store the extracted data
reviews = []

# Variable to store the overall grade
overall_grade = None

# Links
links = [
    'https://flatreviews.com/dubai/reviews/60', #Emirates Hills
    'https://flatreviews.com/dubai/reviews/25', #Jumeirah Islands
    'https://flatreviews.com/dubai/reviews/279', #Meadows 4
    'https://flatreviews.com/dubai/reviews/390', #Meadows 1
    'https://flatreviews.com/dubai/reviews/416', #Lakes - Hattan I
    'https://flatreviews.com/dubai/reviews/49', #Silicon Oasis
    'https://flatreviews.com/dubai/reviews/697', #Meadows 9
    'https://flatreviews.com/dubai/reviews/551', #Meadows 2
    'https://flatreviews.com/dubai/reviews/268', #Meadows 3
    'https://flatreviews.com/dubai/reviews/455', #Meadows 5
    'https://flatreviews.com/dubai/reviews/552', #Meadows 6
    'https://flatreviews.com/dubai/reviews/541', #Meadows 7
    'https://flatreviews.com/dubai/reviews/692', #Meadows 8
    'https://flatreviews.com/dubai/reviews/472', #Lakes - Hattan II
    'https://flatreviews.com/dubai/reviews/99999', #Emaar's Street Of Dreams
    'https://flatreviews.com/dubai/reviews/682', #Arabian Ranches - Saheel 1
    'https://flatreviews.com/dubai/reviews/5898', #Arabian Ranches - Saheel 2
    'https://flatreviews.com/dubai/reviews/5899', #Arabian Ranches - Saheel 3
    'https://flatreviews.com/dubai/reviews/8313', #Arabian Ranches - Saheel 4
    'https://flatreviews.com/dubai/reviews/9049', #Lakes - Hattan III
    'https://flatreviews.com/dubai/reviews/664', #Arabian Ranches - Golf Homes
    'https://flatreviews.com/dubai/reviews/8311', #Al Barari
    'https://flatreviews.com/dubai/reviews/33', #Dubai Sports City
    'https://flatreviews.com/dubai/reviews/5902', #Mirador La Colccion 2
    'https://flatreviews.com/dubai/reviews/395', #Mirador
    'https://flatreviews.com/dubai/reviews/404', #Savannah
    'https://flatreviews.com/dubai/reviews/666', #Terranova
    'https://flatreviews.com/dubai/reviews/266', #Mirador La Colccion 1
    'https://flatreviews.com/dubai/reviews/494', #Alvorada - 1
    'https://flatreviews.com/dubai/reviews/75', #Palm Jumeirah
    'https://flatreviews.com/dubai/reviews/5906', #Arabian Ranches - Alvorada - 4
    'https://flatreviews.com/dubai/reviews/620', #Lakes - Maeen 1
    'https://flatreviews.com/dubai/reviews/7647', #Lakes - Maeen 2
    'https://flatreviews.com/dubai/reviews/7648', #Lakes - Maeen 3
    'https://flatreviews.com/dubai/reviews/7649', #Lakes - Maeen 4
    'https://flatreviews.com/dubai/reviews/99999', #Sama Al Jadaf
    'https://flatreviews.com/dubai/reviews/3984', #TECOM Site C
    'https://flatreviews.com/dubai/reviews/8071', #Jumeirah Golf
    'https://flatreviews.com/dubai/reviews/37', #Jumeirah Park
    'https://flatreviews.com/dubai/reviews/8392', #Arabian Ranches - Polo Homes
    'https://flatreviews.com/dubai/reviews/8066', #Dubai Health Care City Phase 1
    'https://flatreviews.com/dubai/reviews/922', #Dubiotech
    'https://flatreviews.com/dubai/reviews/99999', #Jumeirah Lakes Towers
    'https://flatreviews.com/dubai/reviews/52', #Al Waha Villas
    'https://flatreviews.com/dubai/reviews/99999', #Residential Complex
    'https://flatreviews.com/dubai/reviews/72', #Falcon City
    'https://flatreviews.com/dubai/reviews/52', #Al Waha Villas
    'https://flatreviews.com/dubai/reviews/52', #Al Waha Villas
    'https://flatreviews.com/dubai/reviews/52', #Al Waha Villas
    'https://flatreviews.com/dubai/reviews/52', #Al Waha Villas
    'https://flatreviews.com/dubai/reviews/52', #Al Waha Villas
    'https://flatreviews.com/dubai/reviews/52', #Al Waha Villas
    'https://flatreviews.com/dubai/reviews/52', #Al Waha Villas
    'https://flatreviews.com/dubai/reviews/52', #Al Waha Villas
    
    
    
    
]

# Ratings labels
labels = ['View', 'Finishing', 'Layout and size', 'Garden', 'Noise level', 'Neighbours',
          'Traffic', 'Public transportation', 'Own parking', 'Guest parking', 'Security',
          'Swimming pool', 'Gym', 'Maintenance', 'Nearby shops and restaurants',
          'Walking possibilities', 'Pet friendly', 'Value for money']

In [6]:
counter = 1

for link in links:
    while True:
        url = link + '?page=' + str(counter)
        print(url)
        
        # Send a GET request to the URL
        response = requests.get(url)

        # Create a BeautifulSoup object
        soup = BeautifulSoup(response.text, 'html.parser')
        reviews_blocks = soup.find_all('div', class_='fr-review-module')

        counter += 1 

        if len(reviews_blocks) == 0:
            break


        # Data extraction
        review_texts = soup.find_all('div', class_='fr-review-text show-for-large')
        review_dates = soup.find_all('div', class_='fr-review-date')
        property_name = soup.find('h1', class_='').text.strip()
        reviewer_cases = soup.find_all('div', class_='fr-review-user-stats')
        overall_grades = soup.find_all('div', class_='fr-location-average-score')
        positives_elements = soup.find_all(class_="fr-review-posnegs-label-positive cell large-shrink")
        negatives_elements = soup.find_all(class_="fr-review-posnegs-label-negative cell large-shrink")
        reviews_par = soup.find_all("div", class_="fr-review-module")

        # Extract the overall grade if it's not already extracted
        if overall_grade is None:
            overall_grade_text = overall_grades[0].text.strip() if overall_grades else ""  # Get the overall grade text
            numeric_grade_match = re.search(r'(\d+(\.\d+)?)', overall_grade_text)  # Extract numeric part using regex
            if numeric_grade_match:
                overall_grade = float(numeric_grade_match.group())  # Convert to float

        # Iterate over the review texts, dates, and grades
        for text, date, case, par in zip_longest(review_texts, review_dates, reviewer_cases, reviews_par):
            review_date = date.text.strip().split(',')[1].strip() if date else ""  # Extract the part after the comma for review date
            review_case = case.text.strip().split(',')[1].strip().replace('.', '') if case else "" # Extract it for the reviewer case and remove '.'

            # Extract the text of each positive review
            positive_review_element = positives_elements.pop(0).find_next_sibling()
            positive_review = positive_review_element.text.strip() if positive_review_element else ""
            positive_review = positive_review if positive_review else ""  # Check if positive_review is empty and replace it with "NA"

            # Extract the text of each negative review
            negative_review_element = negatives_elements.pop(0).find_next_sibling()
            negative_review = negative_review_element.text.strip() if negative_review_element else ""
            negative_review = negative_review if negative_review else ""  # Check if negative_review is empty and replace it with "NA"

            # Extract review text and remove unnecessary spaces and newlines
            review_text = re.sub(r'\s+', ' ', text.text.strip()) if text else ""
            # Check if the review text is empty or contains only '-'
            if not review_text or review_text == '-':
                review_text = ""
            # Extract ratings
            ratings = {}
            for label in labels:
                par_label = par.find("label", text=label)
                if par_label is not None:
                    parent_div = par_label.find_parent("div", class_="fr-review-facet")
                    filled_stars = parent_div.select('i.material-icons:not(.star-none)')
                    filled_stars_count = len(filled_stars)
                    ratings[label] = filled_stars_count
                else:
                    ratings[label] = ""


            review = {
                'Property name': property_name,
                'Overall grade': overall_grade,
                'Review date': review_date,
                'Reviewer class': review_case,
                'Positive Review': positive_review,
                'Negative Review': negative_review,
                'Review text': review_text,
                **ratings
            }
            
            reviews.append(review)
    
    counter = 1
        
DF = pd.DataFrame(data = reviews)

https://flatreviews.com/dubai/reviews/60?page=1
https://flatreviews.com/dubai/reviews/60?page=2
https://flatreviews.com/dubai/reviews/25?page=1
https://flatreviews.com/dubai/reviews/25?page=2
https://flatreviews.com/dubai/reviews/279?page=1
https://flatreviews.com/dubai/reviews/390?page=1
https://flatreviews.com/dubai/reviews/390?page=2
https://flatreviews.com/dubai/reviews/416?page=1
https://flatreviews.com/dubai/reviews/49?page=1
https://flatreviews.com/dubai/reviews/49?page=2
https://flatreviews.com/dubai/reviews/49?page=3
https://flatreviews.com/dubai/reviews/49?page=4
https://flatreviews.com/dubai/reviews/697?page=1
https://flatreviews.com/dubai/reviews/697?page=2
https://flatreviews.com/dubai/reviews/551?page=1
https://flatreviews.com/dubai/reviews/268?page=1
https://flatreviews.com/dubai/reviews/268?page=2
https://flatreviews.com/dubai/reviews/455?page=1
https://flatreviews.com/dubai/reviews/455?page=2
https://flatreviews.com/dubai/reviews/552?page=1
https://flatreviews.com/duba

In [7]:
DF

,Property name,Overall grade,Review date,Reviewer class,Positive Review,Negative Review,Review text,View,Finishing,Layout and size,...,Own parking,Guest parking,Security,Swimming pool,Gym,Maintenance,Nearby shops and restaurants,Walking possibilities,Pet friendly,Value for money
0,Emirates Hills,3.0,November 2014,Family with children,Good location,Expensive,,,1,1,...,5,5,4,5,1,1,4,1,4,3
1,Jumeirah Islands,3.0,January 2015,Living with parents,"Private parking, quiet area, good for walking","No supermarkets within the community, no visit...",,5,4,4,...,5,2,4,2,1,1,3,5,3,4
2,Jumeirah Islands,3.0,November 2014,Family with children,The greenry and the feel of the well developed...,The access road to SZR,"An outstanding development by Nakheel, present...",4,4,5,...,5,5,5,5,4,4,5,5,5,5
3,Jumeirah Islands,3.0,November 2014,Family with children,"Spacious, Green",Lack of security and upkeep of the green commo...,We have been living in the Islands for 7 years...,5,4,4,...,5,3,1,4,4,1,4,5,5,3
4,Jumeirah Islands,3.0,July 2014,Family with children,"Community, Club, Sports Facilities",Construction Nearby,Definitely one of the top places to stay in! T...,3,4,4,...,5,5,5,4,5,5,3,4,5,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
236,Jumeirah Park,3.0,November 2014,Family with children,Nice new and bright villas,No areas for kids to play,"The house is new (first residents), bright, ni...",2,4,5,...,5,5,2,5,1,1,2,2,5,3
237,Jumeirah Park,3.0,November 2014,Living with spouse,Peace and layout of villa,Services,A new community developed by Nakheel. The layo...,4,4,5,...,5,5,2,5,1,3,3,5,5,4
238,Jumeirah Park,3.0,July 2014,Family with children,The calmness,The AC issues,"Since I moved here, it feels like home altoget...",3,5,4,...,4,3,4,3,1,3,1,3,3,4
239,Al Waha,3.0,June 2014,Family with children,"Traffic, easy access, no buildings, amazing views","Traffic lights not working, people going in wr...",,5,3,5,...,5,1,5,5,5,5,2,5,5,5


In [51]:
DF.to_csv('./Reviews.csv', index=False)